# Word counts
A [`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) using `SciKit-Learn` is used for extracting features from the data. The features are then used to train a **random forest classifier**.

Submission accuracy: `0.60200`

Download necessary NLTK files:
* `stopwords`: Stopwords Corpus
* `wordnet`: WordNet

In [1]:
%%script false
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [2]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from os.path import join
from tqdm import tqdm

tqdm.pandas()

In [3]:
src = 'data'
df_train = pd.read_csv(join(src, 'labeledTrainData.tsv'), sep='\t')
df_train.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


## Data cleaning
Here we use the data cleaning steps in `1_data_cleaning.ipynb`.

In [4]:
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

def clean_sentence(sentence):
    removed_markup = BeautifulSoup(sentence, 'html.parser').text
    removed_punctuation = re.sub(r'[^a-zA-Z]', ' ', removed_markup)
    tokens = removed_punctuation.lower().split()
    removed_stopwords = [w for w in tokens if w not in stopwords]
    lemmatized = [lemmatizer.lemmatize(w) for w in removed_stopwords]
    return ' '.join(lemmatized)

In [5]:
clean_sentence(df_train['review'][0])

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighty maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle message mj feeling towards press also obvious message drug bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fan would say made fan true really nice actual feature film bit finally start minute excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord want mj dead bad beyond mj overheard plan nah joe pesci character ranted wanted people know supplying drug etc dunno maybe hate mj music lot cool thing like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually director hate working one kid let a

In [6]:
df_train['review_cleaned'] = df_train['review'].progress_apply(clean_sentence)
df_train.head()

100%|██████████| 25000/25000 [00:27<00:00, 912.08it/s] 


,id,sentiment,review,review_cleaned
0,5814_8,1,With all this stuff going down at the moment w...,stuff going moment mj started listening music ...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...",classic war world timothy hines entertaining f...
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,film start manager nicholas bell giving welcom...
3,3630_4,0,It must be assumed that those who praised this...,must assumed praised film greatest filmed oper...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy wondrously unpretentious explo...


## Feature extraction
Extract features from the word counts. The features are the top-`n` words, with their corresponding counts in the data.

In [7]:
vectorizer = CountVectorizer(analyzer='word', max_features=5000)
train_features = vectorizer.fit_transform(df_train['review_cleaned'])
train_features = train_features.toarray()  # from scipy.sparse.csr.csr_matrix
train_features.shape

(25000, 5000)

Examine the word counts for our vocabulary.

In [8]:
vocab = vectorizer.get_feature_names()  # list of words
dist = np.sum(train_features, axis=0)  # np.ndarray of shape (n_words, )
for i in range(5):
    print(vocab[i], dist[i])

abandoned 187
abc 125
ability 562
able 1259
abraham 92


## Training
Use the features obtained from the `CountVectorizer` and fit them with a `RandomForestClassifier`.

The model learns the relationship between the frequencies of words in the vocabulary, and whether such words indicate a positive or negative sentiment.

In [9]:
%%time
clf = RandomForestClassifier(n_estimators=100, random_state=0)
clf.fit(train_features, df_train['sentiment'])

CPU times: user 1min 38s, sys: 208 ms, total: 1min 39s
Wall time: 1min 39s


## Testing
Load and clean the test data similarly, before making predictions with the trained classifier.

In [10]:
df_test = pd.read_csv(join(src, 'testData.tsv'), sep='\t')
df_test.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


In [11]:
df_test['review_cleaned'] = df_test['review'].progress_apply(clean_sentence)
df_test.head()

100%|██████████| 25000/25000 [00:24<00:00, 1034.75it/s]


,id,review,review_cleaned
0,12311_10,Naturally in a film who's main themes are of m...,naturally film main theme mortality nostalgia ...
1,8348_2,This movie is a disaster within a disaster fil...,movie disaster within disaster film full great...
2,5828_4,"All in all, this is a movie for kids. We saw i...",movie kid saw tonight child loved one point ki...
3,7186_2,Afraid of the Dark left me with the impression...,afraid dark left impression several different ...
4,12128_7,A very accurate depiction of small time mob li...,accurate depiction small time mob life filmed ...


In [12]:
test_features = vectorizer.fit_transform(df_test['review_cleaned'])
test_features = test_features.toarray()
test_features.shape

(25000, 5000)

In [13]:
%%time
pred = clf.predict(test_features)

CPU times: user 2.78 s, sys: 196 ms, total: 2.98 s
Wall time: 2.97 s


In [14]:
output = pd.DataFrame({'id': df_test['id'], 'sentiment': pred})
output.head()

,id,sentiment
0,12311_10,1
1,8348_2,0
2,5828_4,0
3,7186_2,0
4,12128_7,1


In [15]:
output.shape

(25000, 2)

In [16]:
output.to_csv('submission/wordcount_randomforest.csv', index=False)